In [1]:
import tensorflow as tf
import requests
import pandas as pd
import numpy as np

2022-08-02 07:53:39.330746: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-02 07:53:39.330810: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaPrevisaoTeste"

In [3]:
r = requests.get(url)
j = r.json()
dfTreino = pd.DataFrame.from_dict(j)

In [4]:
dfTreino.head()

id  grupoAcaoId  periodoTreinoRedeId  tipoExemploTreinoId  regraProjecaoId  \
0   2           12                    1                    1               30   
1   8           12                    1                    1               32   

   redeNeuralId  ativoPrevisao  limiteParaEntrada tipoCompraVenda  
0             2              1                0.5               V  
1             2              1                0.3               C

In [5]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [6]:
def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoPorDiaB3?diaNum=" + str(diaCorrente) +"&idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    #print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    #print(dfEntradas.head())
    return dfEntradas

In [7]:
def trataRede():
    treino = treinoCorrente
    #print('ID:' , treino['id'])
    nome_arquivo = 'pesos/treino_rede' + str(treino['id']) + '.h5'
    #print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    #modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(treino['id'])
    dfDados['tipoCompraVenda'] = treino['tipoCompraVenda']
    dfEntradas = incluiEntrada(treino)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    #print('ticker previsão:' , len(dfPrevisao))
    json = dfPrevisao.to_dict(orient='list')
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoTestes/recebePrevisaoTreinoTeste'
    #print(json)
    response = requests.post(urlPrevisao, data=json)
    #print(response)

In [8]:
def trataDiaPregao(dia):
    print('Tratando Dia: ' , dia.diaNum)
    global diaCorrente
    diaCorrente = dia.diaNum
    urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDiaNum?diaNum={}".format(dia.diaNum)
    #print(urlExemplos)
    r = requests.get(urlExemplos)
    j = r.json()
    global dfDados
    dfDados = pd.DataFrame.from_dict(j)
    #print(dfDados.head())
    dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
    global X
    X = np.stack(dfTratadoX.values)
    X.shape
    dfDados.drop('campoX', inplace=True, axis=1)
    trataRede()

In [9]:
def trataTreinoRede(treino):
    #print('Tratando Rede: ' , treino.id)
    global treinoCorrente
    treinoCorrente = treino
    #print('TreinoCorrente: ' , treinoCorrente)
    urlDias = "http://vps-40d69db1.vps.ovh.ca:23004/api/DiaPregaos/obtemPeriodoPrevisaoTeste?idPeriodoTreinoRede={}".format(treino.periodoTreinoRedeId)
    r = requests.get(urlDias)
    j = r.json()
    dfDias = pd.DataFrame.from_dict(j)
    #print(dfDias.head())
    dfDias.apply(trataDiaPregao, axis='columns')

In [ ]:
dfTreino.apply(trataTreinoRede, axis='columns')

Tratando Dia:  20220601


2022-08-02 07:54:10.277606: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-02 07:54:10.286552: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-08-02 07:54:10.497956: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-02 07:54:14.951209: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12960000 exceeds 10% of free system memory.
2022-08-02 07:54:15.340426: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12960000 exceeds 10% of free system memory.
2022-08-02 07:54:

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1800)              3241800   
                                                                 
 dense_1 (Dense)             (None, 1400)              2521400   
                                                                 
 dense_2 (Dense)             (None, 1000)              1401000   
                                                                 
 dense_3 (Dense)             (None, 600)               600600    
                                                                 
 dense_4 (Dense)             (None, 200)               120200    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 7,885,201
Trainable params: 7,885,201
Non-

Tratando Dia:  20220609
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1800)              3241800   
                                                                 
 dense_1 (Dense)             (None, 1400)              2521400   
                                                                 
 dense_2 (Dense)             (None, 1000)              1401000   
                                                                 
 dense_3 (Dense)             (None, 600)               600600    
                                                                 
 dense_4 (Dense)             (None, 200)               120200    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 7,885,201
Trainabl

Tratando Dia:  20220621
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1800)              3241800   
                                                                 
 dense_1 (Dense)             (None, 1400)              2521400   
                                                                 
 dense_2 (Dense)             (None, 1000)              1401000   
                                                                 
 dense_3 (Dense)             (None, 600)               600600    
                                                                 
 dense_4 (Dense)             (None, 200)               120200    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 7,885,201
Trainabl